In [20]:
import pandas as pd
import numpy as np

df = pd.read_json('../data/raw/full_property.json')

In [21]:
# take a look at the size of the data
df.shape

(12, 11206)

In [22]:
# there are 12 rows in the data, but we have 12 features,
# so data needs to have 12 columns instead -- need to transpose this data
df = df.transpose()
df.head()

,name,cost_text,coordinates,Bed,Bath,Park,property_type,desc_head,desc,additional features,internal_area_sqkm,land_area_sqkm
https://www.domain.com.au/unit-503-466-swanston-street-carlton-vic-3053-16066115,unit 503/466 Swanston Street Carlton VIC 3053,$320,"[-37.8059838, 144.9632701]",1,1,0,Apartment / Unit / Flat,Fully Furnished Studio Apartment in Swanston,"class=""css-dxogle"">* Unverified feature<svg a...","[Alarm System, Built in wardrobes, Intercom]",[],[]
https://www.domain.com.au/61-rowans-road-highett-vic-3190-16080336,61 Rowans Road Highett VIC 3190,$460.00,"[-37.9475765, 145.0509956]",3,1,1,House,Quaint three bedroom in a great location!,"class=""css-dxogle"">* Unverified feature<svg a...","[Split System Air Con, Split System Heating, E...",NaN,[]
https://www.domain.com.au/29-caspian-circuit-point-cook-vic-3030-16023583,29 Caspian Circuit Point Cook VIC 3030,$430.00 per week,"[-37.9002294, 144.7452156]",3,2,2,House,Life at Your Fingertips,"class=""css-dxogle"">* Unverified feature<svg a...",[],NaN,[]
https://www.domain.com.au/36-bradleys-lane-north-warrandyte-vic-3113-16038589,36 Bradleys Lane North Warrandyte VIC 3113,$720 per week,"[-37.7370336, 145.2124356]",4,3,3,House,Bradleys Lane,"class=""css-dxogle"">* Unverified feature<svg a...",[],NaN,[]
https://www.domain.com.au/12-isaac-road-keysborough-vic-3173-16020786,12 Isaac Road Keysborough VIC 3173,$515 pw,"[-37.9840266, 145.1782469]",3,3,1,House,LUXURY SUBURBAN LIVING,"class=""css-dxogle"">* Unverified feature<svg a...","[Built in wardrobes, Dishwasher]",NaN,[]


### 1: Cost

In [23]:
# the cost of a real estate property is in dollars per week.
# I will transform all the values in the cost_text column into integers.

df2 = df.copy()
import re

for i in range(df2.shape[0]):
    cost = re.findall(r"[\d,]+", df2.iat[i, 1])

    if len(cost) == 0:
        df2.iat[i, 1] = np.nan

    else:
        # find the element with length 3, since it's the most likely one
        cost2 = [i for i in cost if len(i) == 3]

        if len(cost2) == 1:
            # if there's only one, then it's very likely that it's the correct price we want
            df2.iat[i, 1] = int(cost2[0])

        else:
            # otherwise, this is where things get wacky
            # might have to clean the data manually
            
            # some possibilities: really cheap price ($60 per week),
            # really expensive price ($1000+ per week),
            # only the monthly price is listed, while the weekly price is unknown,
            # only the annual price is listed, while the weekly price is unknown,
            # some other gibberish text that contains numbers that isn't the weekly price.
            #print(df2.iat[i, 1])
            #print(cost)
            #print("")
            pass
        
            # TODO: after the real estate data is finalised, we can either clean this part manually, or simply ditch it

### 2: Description

In [24]:
# might need to install these libraries first
import enchant   
import nltk
from nltk.stem import PorterStemmer

def preprocess_text(text):
    # preprocesses a string of texts :)
    
    # step 1: remove non-alphabetic characters
    text = re.sub(r'[^a-z\sA-Z]', ' ', text)
    # step 2: convert all spacing characters such as tabs and newlines to 
    # whitespace and ensure that only one whitespace character exists between each word
    text = re.sub(r'\s+', ' ', text)
    # step 3: Change all uppercase characters to lower case
    text = text.lower()
    # step 4: tokenise into words
    words = text.split()
    # step 5: remove invalid English words and single-letter words, and then stemming
    d = enchant.Dict('en')
    ps = PorterStemmer()
    words = [ps.stem(i) for i in words if(d.check(i) == True) and (len(i) > 1)]
    return words

In [25]:
df3 = df2.copy()


for i in range(df3.shape[0]):
    # there are many instances with invalid descriptions, 
    # and fortunately, they seem relatively easy to find.
    invalid = re.findall(r'class="', df3.iat[i, 8])
    if invalid:
        df3.iat[i, 8] = np.nan

    # text preprocessing
    # I'm not sure how useful this "desc" attribute is going to be,
    # so if you wanna use this attribute, just uncomment this part of the code
    # just note that it will take a while to run
    #else:
    #    df3.iat[i, 8] = preprocess_text(df3.iat[i, 8])

#df3.head(3)
df3.where(df3['coordinates'] == "Not Listed").count()

name                   13
cost_text               0
coordinates            13
Bed                    13
Bath                   13
Park                   13
property_type          13
desc_head              13
desc                   11
additional features    13
internal_area_sqkm      0
land_area_sqkm         13
dtype: int64

### Final 3 features

In [26]:
# only need to replace empty list with NaN values
for i in range(df3.shape[0]):
    try:
        # there is definitely a better way to do this than try except, but this works so
        if len(df3.iat[i, -1]) == 0:
            df3.iat[i, -1] = np.nan
    except:
        pass
    if len(df3.iat[i, -2]) == 0:
        df3.iat[i, -2] = np.nan
    if len(df3.iat[i, -3]) == 0:
        df3.iat[i, -3] = np.nan

df3.head()

TypeError: object of type 'float' has no len()

Before saving the dataframe to csv, additional preprocessing might be required

In [39]:
# save to csv
filename = "full_property_data"
df3.to_csv(filename)

ValueError: Invalid file path or buffer object type: <class 'ellipsis'>